In [1]:
import pandas as pd
import numpy as np
import datetime
import sqlite3
import matplotlib.pyplot as plt
%matplotlib inline 
from random import sample

In [2]:
path = '../data/'

### Slide `pickup_taxizone_id` column up by 1 within key (`hack_license`, `medallion`)

In [3]:
# path = '../data/'
# conn = sqlite3.connect(path + 'trip_data.db')

# q_license = """
# SELECT distinct(hack_license) as UNIQ_LIC
# FROM trip
# """

# unique_license = pd.read_sql(q_license, conn)

# LIC_SAMPLE_NUM = 10
# sampled_drivers = sample(unique_license.UNIQ_LIC.tolist(), LIC_SAMPLE_NUM)

# # find the pickup and dropoff time of next trip
# q_trip = """
# SELECT dropoff_datetime, dropoff_taxizone_id, 
#         lag(pickup_taxizone_id, -1, NULL) over (partition by hack_license, medallion order by pickup_datetime) as pickup_taxizone_id_next,
#         lag(pickup_datetime, -1, NULL) over (partition by hack_license order by pickup_datetime) as pickup_datetime_next
#     FROM trip
#     WHERE hack_license in {0}
# ORDER BY hack_license ASC, medallion ASC, pickup_datetime ASC, dropoff_datetime ASC
# """

# trip_df = pd.read_sql(q_trip.format(tuple(sampled_drivers)), conn)
# conn.close()

In [4]:
# path = '../data/'
# conn = sqlite3.connect(path + 'trip_data.db')

# # find the pickup and dropoff time of next trip
# q_trip = """
# SELECT dropoff_datetime, dropoff_taxizone_id, 
#         lag(pickup_taxizone_id, -1, NULL) over (partition by hack_license, medallion order by pickup_datetime) as pickup_taxizone_id_next,
#         lag(pickup_datetime, -1, NULL) over (partition by hack_license order by pickup_datetime) as pickup_datetime_next
#     FROM trip
# ORDER BY hack_license ASC, medallion ASC, pickup_datetime ASC, dropoff_datetime ASC
# LIMIT 10
# """

# trip_df = pd.read_sql(q_trip, conn)
# conn.close()

### DF processing

* Load cleaned trip df
* shift pickup time and taxi zone 1 row over
* keep only next pickup = current dropoff zone

In [5]:
cleaned_trip_df = pd.read_csv('../data/trip_cleaned.csv')



In [6]:
trip_df = cleaned_trip_df[['hack_license', 'pickup_taxizone_id', 'dropoff_taxizone_id',
                           'pickup_datetime', 'dropoff_datetime']]

In [7]:
trip_df

,hack_license,pickup_taxizone_id,dropoff_taxizone_id,pickup_datetime,dropoff_datetime
0,2013000001,148.0,79.0,2013-06-24 18:55:00,2013-06-24 19:01:00
1,2013000001,164.0,143.0,2013-06-24 19:12:00,2013-06-24 19:30:00
2,2013000001,142.0,79.0,2013-06-24 19:45:00,2013-06-24 20:10:00
3,2013000001,234.0,141.0,2013-06-24 20:17:00,2013-06-24 20:28:00
4,2013000001,141.0,263.0,2013-06-24 20:30:00,2013-06-24 20:35:00
...,...,...,...,...,...
13885734,2013038800,45.0,79.0,2013-06-30 22:10:00,2013-06-30 22:18:00
13885735,2013038800,144.0,65.0,2013-06-30 22:29:00,2013-06-30 22:38:00
13885736,2013038800,144.0,13.0,2013-06-30 22:53:00,2013-06-30 23:00:00
13885737,2013038800,45.0,148.0,2013-06-30 23:08:00,2013-06-30 23:17:00


In [8]:
trip_df.sort_values(['hack_license', 'pickup_datetime'], inplace=True)
trip_df['pickup_datetime_next'] = trip_df.groupby(['hack_license'])['pickup_datetime'].shift(-1)
trip_df['pickup_taxizone_id_next'] = trip_df.groupby(['hack_license'])['pickup_taxizone_id'].shift(-1)

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user

In [9]:
trip_df = trip_df.loc[trip_df['dropoff_taxizone_id'] == trip_df['pickup_taxizone_id_next']]
trip_df.dropna(subset = ['pickup_taxizone_id_next'], inplace=True)

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
trip_df

,hack_license,pickup_taxizone_id,dropoff_taxizone_id,pickup_datetime,dropoff_datetime,pickup_datetime_next,pickup_taxizone_id_next
3,2013000001,234.0,141.0,2013-06-24 20:17:00,2013-06-24 20:28:00,2013-06-24 20:30:00,141.0
4,2013000001,141.0,263.0,2013-06-24 20:30:00,2013-06-24 20:35:00,2013-06-24 20:37:00,263.0
5,2013000001,263.0,239.0,2013-06-24 20:37:00,2013-06-24 20:49:00,2013-06-24 20:52:00,239.0
8,2013000001,162.0,233.0,2013-06-26 17:24:00,2013-06-26 17:35:00,2013-06-26 17:36:00,233.0
12,2013000001,233.0,114.0,2013-06-26 18:28:00,2013-06-26 18:50:00,2013-06-26 18:53:00,114.0
...,...,...,...,...,...,...,...
13885719,2013038799,224.0,224.0,2013-06-30 19:49:31,2013-06-30 19:55:13,2013-06-30 21:12:41,224.0
13885720,2013038799,224.0,224.0,2013-06-30 21:12:41,2013-06-30 21:30:09,2013-06-30 21:31:35,224.0
13885721,2013038799,224.0,224.0,2013-06-30 21:31:35,2013-06-30 21:47:59,2013-06-30 21:59:27,224.0
13885722,2013038799,224.0,224.0,2013-06-30 21:59:27,2013-06-30 22:21:20,2013-06-30 22:24:04,224.0


### Compute cruise time

In [11]:
trip_df['pickup_datetime_next'] = pd.to_datetime(trip_df['pickup_datetime_next'])
trip_df['dropoff_datetime'] = pd.to_datetime(trip_df['dropoff_datetime'])

trip_df['cruise_time'] = (trip_df['pickup_datetime_next'] - trip_df['dropoff_datetime'])/np.timedelta64(1, 'm')
trip_df.reset_index(inplace=True)


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

In [12]:
k = 5
print('Cruise Time less than 0 minutes: {}'.format(trip_df.loc[(trip_df['cruise_time'] < 0)].shape[0]))
print('Cruise Time greater than {} minutes: {}'.format(k*60, trip_df.loc[(trip_df['cruise_time'] <= k*60)].shape[0]))
print('These records are dropped')

Cruise Time less than 0 minutes: 525
Cruise Time greater than 300 minutes: 4974616
These records are dropped


In [13]:
trip_df.shape


(4990678, 9)

In [14]:
trip_df.head()


,index,hack_license,pickup_taxizone_id,dropoff_taxizone_id,pickup_datetime,dropoff_datetime,pickup_datetime_next,pickup_taxizone_id_next,cruise_time
0,3,2013000001,234.0,141.0,2013-06-24 20:17:00,2013-06-24 20:28:00,2013-06-24 20:30:00,141.0,2.0
1,4,2013000001,141.0,263.0,2013-06-24 20:30:00,2013-06-24 20:35:00,2013-06-24 20:37:00,263.0,2.0
2,5,2013000001,263.0,239.0,2013-06-24 20:37:00,2013-06-24 20:49:00,2013-06-24 20:52:00,239.0,3.0
3,8,2013000001,162.0,233.0,2013-06-26 17:24:00,2013-06-26 17:35:00,2013-06-26 17:36:00,233.0,1.0
4,12,2013000001,233.0,114.0,2013-06-26 18:28:00,2013-06-26 18:50:00,2013-06-26 18:53:00,114.0,3.0


### Remove records with cruise time > 300 mins or <0 

In [15]:
trip_df = trip_df.loc[(trip_df['cruise_time'] <= k*60) &
                                        (trip_df['cruise_time'] > 0)]



## Round DO time to 15 minute and 1 hour interval

In [16]:
trip_df_15m = trip_df.copy()
trip_df_60m = trip_df.copy()


In [17]:
trip_df_15m['dropoff_datetime_interval'] = trip_df_15m['dropoff_datetime'].dt.round('15min').dt.time
trip_df_60m['dropoff_datetime_interval'] = trip_df_60m['dropoff_datetime'].dt.round('1h').dt.time


## [aside] Coversion table 

In [18]:
time_index_15m = []
time_index_1h = []
time_interval = []
m_index = 0
h_index = 0
for h in range(24):
    for q in range(4):
        time_interval.append(datetime.time(h, q*15))
        time_index_1h.append(h_index)
        time_index_15m.append(m_index)
        m_index += 1
    h_index += 1
       

In [19]:
interval_index_table = pd.DataFrame({'interval': time_interval, 'index_15m': time_index_15m, 'index_60m': time_index_1h})
interval_index_table

,interval,index_15m,index_60m
0,00:00:00,0,0
1,00:15:00,1,0
2,00:30:00,2,0
3,00:45:00,3,0
4,01:00:00,4,1
...,...,...,...
91,22:45:00,91,22
92,23:00:00,92,23
93,23:15:00,93,23
94,23:30:00,94,23


In [20]:
interval_index_table.to_csv('interval_index_table_0.csv', index=False)


In [21]:
conversion_15m = dict(zip(interval_index_table.interval, interval_index_table.index_15m))
conversion_60m = dict(zip(interval_index_table.interval, interval_index_table.index_60m))


## convert interval to index

In [22]:
trip_df_15m['dropoff_datetime_index'] = [conversion_15m[t] for t in trip_df_15m['dropoff_datetime_interval']]
trip_df_60m['dropoff_datetime_index'] = [conversion_60m[t] for t in trip_df_60m['dropoff_datetime_interval']]


In [23]:
trip_df_15m.head()


,index,hack_license,pickup_taxizone_id,dropoff_taxizone_id,pickup_datetime,dropoff_datetime,pickup_datetime_next,pickup_taxizone_id_next,cruise_time,dropoff_datetime_interval,dropoff_datetime_index
0,3,2013000001,234.0,141.0,2013-06-24 20:17:00,2013-06-24 20:28:00,2013-06-24 20:30:00,141.0,2.0,20:30:00,82
1,4,2013000001,141.0,263.0,2013-06-24 20:30:00,2013-06-24 20:35:00,2013-06-24 20:37:00,263.0,2.0,20:30:00,82
2,5,2013000001,263.0,239.0,2013-06-24 20:37:00,2013-06-24 20:49:00,2013-06-24 20:52:00,239.0,3.0,20:45:00,83
3,8,2013000001,162.0,233.0,2013-06-26 17:24:00,2013-06-26 17:35:00,2013-06-26 17:36:00,233.0,1.0,17:30:00,70
4,12,2013000001,233.0,114.0,2013-06-26 18:28:00,2013-06-26 18:50:00,2013-06-26 18:53:00,114.0,3.0,18:45:00,75


In [24]:
trip_df_60m.head()


,index,hack_license,pickup_taxizone_id,dropoff_taxizone_id,pickup_datetime,dropoff_datetime,pickup_datetime_next,pickup_taxizone_id_next,cruise_time,dropoff_datetime_interval,dropoff_datetime_index
0,3,2013000001,234.0,141.0,2013-06-24 20:17:00,2013-06-24 20:28:00,2013-06-24 20:30:00,141.0,2.0,20:00:00,20
1,4,2013000001,141.0,263.0,2013-06-24 20:30:00,2013-06-24 20:35:00,2013-06-24 20:37:00,263.0,2.0,21:00:00,21
2,5,2013000001,263.0,239.0,2013-06-24 20:37:00,2013-06-24 20:49:00,2013-06-24 20:52:00,239.0,3.0,21:00:00,21
3,8,2013000001,162.0,233.0,2013-06-26 17:24:00,2013-06-26 17:35:00,2013-06-26 17:36:00,233.0,1.0,18:00:00,18
4,12,2013000001,233.0,114.0,2013-06-26 18:28:00,2013-06-26 18:50:00,2013-06-26 18:53:00,114.0,3.0,19:00:00,19


In [25]:
# trip_df_15m.to_csv('../data/trip_df_15m.csv', index=False)

In [27]:
cruise_time_15m = trip_df_15m.groupby(['dropoff_datetime_index', 'dropoff_datetime_interval', 'dropoff_taxizone_id'], as_index=False)['cruise_time'].agg(['mean', 'median', 'count']).reset_index()
cruise_time_15m.rename(columns = {'dropoff_taxizone_id': 'taxizone_id',
                                  'mean': 'avg_cruise_time',
                                  'median': 'med_cruise_time'}, inplace=True)


In [28]:
# penalty = 1000

# cruise_time_15m = cruise_time_15m.pivot_table(index=['dropoff_datetime_index', 'dropoff_datetime_interval'], 
#                            columns= 'taxizone_id', fill_value=penalty)
# cruise_time_15m = cruise_time_15m.stack().reset_index()


In [29]:
cruise_time_15m


,dropoff_datetime_index,dropoff_datetime_interval,taxizone_id,avg_cruise_time,med_cruise_time,count
0,0,00:00:00,2.0,0.875000,0.875000,2
1,0,00:00:00,3.0,5.000000,5.000000,1
2,0,00:00:00,4.0,2.438902,2.000000,419
3,0,00:00:00,7.0,5.526792,3.308333,400
4,0,00:00:00,10.0,8.472222,1.500000,3
...,...,...,...,...,...,...
14589,95,23:45:00,258.0,100.041667,100.041667,2
14590,95,23:45:00,260.0,4.306790,2.150000,27
14591,95,23:45:00,261.0,2.543390,2.000000,237
14592,95,23:45:00,262.0,3.459175,2.191667,416


In [30]:
cruise_time_60m = trip_df_60m.groupby(['dropoff_datetime_index', 'dropoff_datetime_interval', 'dropoff_taxizone_id'], as_index=False)['cruise_time'].agg(['mean', 'median', 'count']).reset_index()
cruise_time_60m.rename(columns = {'dropoff_taxizone_id': 'taxizone_id',
                                  'mean': 'avg_cruise_time',
                                  'median': 'med_cruise_time'}, inplace=True)

In [31]:
# cruise_time_60m = cruise_time_60m.pivot_table(index=['dropoff_datetime_index', 'dropoff_datetime_interval'], 
#                            columns= 'taxizone_id', fill_value=penalty)
# cruise_time_60m = cruise_time_60m.stack().reset_index()


In [32]:
cruise_time_60m


,dropoff_datetime_index,dropoff_datetime_interval,taxizone_id,avg_cruise_time,med_cruise_time,count
0,0,00:00:00,2.0,1.454167,1.475000,4
1,0,00:00:00,3.0,5.000000,5.000000,1
2,0,00:00:00,4.0,2.570164,2.000000,1811
3,0,00:00:00,6.0,40.000000,40.000000,1
4,0,00:00:00,7.0,5.380166,3.100000,1642
...,...,...,...,...,...,...
4592,23,23:00:00,259.0,3.708333,3.708333,2
4593,23,23:00:00,260.0,5.286082,3.000000,97
4594,23,23:00:00,261.0,2.695562,2.000000,1029
4595,23,23:00:00,262.0,3.452740,2.300000,2007


## Export files

In [33]:
cruise_time_15m.to_csv(path + 'cruise_time_15m.csv', index=False)
cruise_time_60m.to_csv(path + 'cruise_time_60m.csv', index=False)


In [34]:
cruise_time_60m.loc[cruise_time_60m['taxizone_id'] == 9]['count'].values#.shape[0]

array([1, 2, 1, 1, 2, 1])

## ====